In [55]:
!python3 -m pip install mediapipe

Defaulting to user installation because normal site-packages is not writeable


In [56]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [57]:
#@markdown To better demonstrate the Pose Landmarker API, we have created a set of visualization tools that will be used in this colab. These will draw the landmarks on a detect person, as well as the expected connections between those markers.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


def draw_pose_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [58]:
import mediapipe as mp 
from mediapipe.tasks import python 
from mediapipe.tasks.python import vision
import cv2
import matplotlib.pyplot as plt

In [59]:
model_pose_path = './tasks/pose_landmarker_heavy.task'
model_hand_path = './tasks/hand_landmarker.task'

In [60]:
import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
VisionRunningMode = mp.tasks.vision.RunningMode

## CRIACAO DE POSE LANDMARKER

PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions

# Create a pose landmarker instance with the video mode:
options_pose = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_pose_path),
    running_mode=VisionRunningMode.VIDEO)

## CRIACAO DE HAND LANDMARKER

HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions

# Create a hand landrmarker instance with the video mode:
options_hand = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_hand_path),
    running_mode=VisionRunningMode.VIDEO)

i=1

## INICIALIAZACAO E USO DOS LANDMARKERS
with PoseLandmarker.create_from_options(options_pose) as pose_landmarker:
  with HandLandmarker.create_from_options(options_hand) as hand_landmarker:
    # The landmarker is initialized. Use it here.
    video_path = "./DB/video-homem-1.mp4"
    cap = cv2.VideoCapture(video_path)

    fps = cap.get(cv2.CAP_PROP_FPS)

    annotated_frames = []
    while cap.isOpened():
      ret, frame = cap.read()
      if not ret:
        break

      rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

      mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

      frame_index = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  # Current frame number
      frame_timestamp_ms = int((frame_index / fps) * 1000)
      # Perform pose landmarking on the provided single image.
      # The pose landmarker must be created with the video mode.
      pose_landmarker_result = pose_landmarker.detect_for_video(mp_image, frame_timestamp_ms)
      
      hand_landmarker_result = hand_landmarker.detect_for_video(mp_image, frame_timestamp_ms)

      aux_image = draw_pose_landmarks_on_image(mp_image.numpy_view(), pose_landmarker_result)
      annotated_image = draw_hand_landmarks_on_image(aux_image, hand_landmarker_result)

      annotated_frames.append(annotated_image)
      #print(f"Pose {i}:", pose_landmarker_result)
      #i += 1

In [61]:
fps

29.97002997002997

In [62]:
import cv2

# Parameters for the video
output_path = "output_video.mp4"  # Path to save the output video
fps = 30  # Frames per second for the output video
frame_height, frame_width, _ = annotated_frames[0].shape  # Height and width from the first frame

# Define the codec and create VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Write each frame to the video
for frame in annotated_frames:
    out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # Convert RGB to BGR for OpenCV

# Release the VideoWriter
out.release()

print(f"Video saved as {output_path}")


Video saved as output_video.mp4
